# Train File
This is a copycat of original `train.py` file, with simple notebook-oriented adaptation, for faster debug work.

## 1. Some libraries if necessary

In [1]:
# !pip install horovod
# !pip install mpi4py

## 2. Import libraries

In [2]:
# Copyright (c) 2020 Uber Technologies, Inc.
# See the License for the specific language governing permissions and
# limitations under the License.

import os

os.umask(0)
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
import argparse
import numpy as np
import random
import sys
import time
import shutil
from importlib import import_module
from numbers import Number

from tqdm.notebook import tqdm
import torch
from torch.utils.data import Sampler, DataLoader
import horovod.torch as hvd


from torch.utils.data.distributed import DistributedSampler

from utils import Logger, load_pretrain

from mpi4py import MPI

## 3. Initialization

In [3]:
comm = MPI.COMM_WORLD
hvd.init()
torch.cuda.set_device(hvd.local_rank())

root_path = os.path.dirname(os.path.abspath(''))
sys.path.insert(0, root_path)


parser = argparse.ArgumentParser(description="Fuse Detection in Pytorch")
parser.add_argument(
    "-m", "--model", default="lanegcn", type=str, metavar="MODEL", help="model name"
)
parser.add_argument("--eval", action="store_true")
parser.add_argument(
    "--resume", default="", type=str, metavar="RESUME", help="checkpoint path"
)
parser.add_argument(
    "--weight", default="/home/jinwei/wise/LaneGCN/results/lanegcn/17.000.ckpt", type=str, metavar="WEIGHT", help="checkpoint path"
)

_StoreAction(option_strings=['--weight'], dest='weight', nargs=None, const=None, default='/home/jinwei/wise/LaneGCN/results/lanegcn/17.000.ckpt', type=<class 'str'>, choices=None, help='checkpoint path', metavar='WEIGHT')

## 4. Support Functions

In [4]:
def worker_init_fn(pid):
    np_seed = hvd.rank() * 1024 + int(pid)
    np.random.seed(np_seed)
    random_seed = np.random.randint(2 ** 32 - 1)
    random.seed(random_seed)


def train(epoch, config, train_loader, net, loss, post_process, opt, val_loader=None):
    train_loader.sampler.set_epoch(int(epoch))
    net.train()
    num_batches = len(train_loader)
    epoch_per_batch = 1.0 / num_batches
    save_iters = int(np.ceil(config["save_freq"] * num_batches))
#     display_iters = int(
#         config["display_iters"] / (hvd.size() * config["batch_size"])
#     )
    display_iters = 368
    print('display_iters: ' +str(display_iters))
    val_iters = int(config["val_iters"] / (hvd.size() * config["batch_size"]))

    start_time = time.time()
    metrics = dict()
    for i, data in tqdm(enumerate(train_loader), total=num_batches, disable=hvd.rank()):
        epoch += epoch_per_batch
        data = dict(data)

        output = net(data)
        loss_out = loss(output, data)
        post_out = post_process(output, data)
        post_process.append(metrics, loss_out, post_out)

        opt.zero_grad()
        loss_out["loss"].backward()
        lr = opt.step(epoch)

        num_iters = int(np.round(epoch * num_batches))
#         print('num_iters: '+str(num_iters))
        if hvd.rank() == 0 and (
            num_iters % save_iters == 0 or epoch >= config["num_epochs"]
        ):
            save_ckpt(net, opt, config["save_dir"], epoch)

        if num_iters % display_iters == 0:
            dt = time.time() - start_time
            metrics = sync(metrics)
            if hvd.rank() == 0:
                post_process.display(metrics, dt, epoch, lr)
            start_time = time.time()
            metrics = dict()

        if num_iters % val_iters == 0:
            val(config, val_loader, net, loss, post_process, epoch)

        if epoch >= config["num_epochs"]:
            val(config, val_loader, net, loss, post_process, epoch)
            return


def val(config, data_loader, net, loss, post_process, epoch):
    net.eval()

    start_time = time.time()
    metrics = dict()
    for i, data in enumerate(data_loader):
        data = dict(data)
        with torch.no_grad():
            output = net(data)
            loss_out = loss(output, data)
            post_out = post_process(output, data)
            post_process.append(metrics, loss_out, post_out)

    dt = time.time() - start_time
    metrics = sync(metrics)
    if hvd.rank() == 0:
        post_process.display(metrics, dt, epoch)
    net.train()


def save_ckpt(net, opt, save_dir, epoch):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    state_dict = net.state_dict()
    for key in state_dict.keys():
        state_dict[key] = state_dict[key].cpu()

    save_name = "%3.3f.ckpt" % epoch
    torch.save(
        {"epoch": epoch, "state_dict": state_dict, "opt_state": opt.opt.state_dict()},
        os.path.join(save_dir, save_name),
    )


def sync(data):
    data_list = comm.allgather(data)
    data = dict()
    for key in data_list[0]:
        if isinstance(data_list[0][key], list):
            data[key] = []
        else:
            data[key] = 0
        for i in range(len(data_list)):
            data[key] += data_list[i][key]
    return data

## 5. Main program

In [5]:
seed = hvd.rank()
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Import all settings for experiment.
args = parser.parse_args(args=[])
# print(args)
model = import_module(args.model)
config, Dataset, collate_fn, net, loss, post_process, opt = model.get_model()
# print(net)

if config["horovod"]:
    opt.opt = hvd.DistributedOptimizer(
        opt.opt, named_parameters=net.named_parameters()
    )

if args.resume or args.weight:
    ckpt_path = args.resume or args.weight
    if not os.path.isabs(ckpt_path):
        ckpt_path = os.path.join(config["save_dir"], ckpt_path)
    ckpt = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
    load_pretrain(net, ckpt["state_dict"])
    if args.resume:
        config["epoch"] = ckpt["epoch"]
        opt.load_state_dict(ckpt["opt_state"])

if args.eval:
    # Data loader for evaluation
    dataset = Dataset(config["val_split"], config, train=False)
    val_sampler = DistributedSampler(
        dataset, num_replicas=hvd.size(), rank=hvd.rank()
    )
    val_loader = DataLoader(
        dataset,
        batch_size=config["val_batch_size"],
        num_workers=config["val_workers"],
        sampler=val_sampler,
        collate_fn=collate_fn,
        pin_memory=True,
    )

    hvd.broadcast_parameters(net.state_dict(), root_rank=0)
    val(config, val_loader, net, loss, post_process, 999)
else:
    # Create log and copy all code
    save_dir = config["save_dir"]
    log = os.path.join(save_dir, "log")
    if hvd.rank() == 0:
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        sys.stdout = Logger(log)

        src_dirs = [root_path]
        dst_dirs = [os.path.join(save_dir, "files")]
        for src_dir, dst_dir in zip(src_dirs, dst_dirs):
            files = [f for f in os.listdir(src_dir) if f.endswith(".py")]
            if not os.path.exists(dst_dir):
                os.makedirs(dst_dir)
            for f in files:
                shutil.copy(os.path.join(src_dir, f), os.path.join(dst_dir, f))

    # Data loader for training
    dataset = Dataset(config["train_split"], config, train=True)
    train_sampler = DistributedSampler(
        dataset, num_replicas=hvd.size(), rank=hvd.rank()
    )
    train_loader = DataLoader(
        dataset,
        batch_size=config["batch_size"],
        num_workers=config["workers"],
        sampler=train_sampler,
        collate_fn=collate_fn,
        pin_memory=True,
        worker_init_fn=worker_init_fn,
        drop_last=True,
    )
    print("Data loader for training done.")

    # Data loader for evaluation
    dataset = Dataset(config["val_split"], config, train=False)
    val_sampler = DistributedSampler(dataset, num_replicas=hvd.size(), rank=hvd.rank())
    val_loader = DataLoader(
        dataset,
        batch_size=config["val_batch_size"],
        num_workers=config["val_workers"],
        sampler=val_sampler,
        collate_fn=collate_fn,
        pin_memory=True,
    )
    print("Data loader for evaluation done.")


    hvd.broadcast_parameters(net.state_dict(), root_rank=0)
    hvd.broadcast_optimizer_state(opt.opt, root_rank=0)

    epoch = config["epoch"]
    print("epoch number: " +str(epoch))
    remaining_epochs = int(np.ceil(config["num_epochs"] - epoch))
    print("remaining epoch number: " +str(remaining_epochs))
    for i in range(remaining_epochs):
        print('epoch number: '+str(i))
        train(epoch + i, config, train_loader, net, loss, post_process, opt, val_loader)

Data loader for training done.


  0%|          | 0/368 [00:00<?, ?it/s]

Data loader for evaluation done.
epoch number: 0
remaining epoch number: 36
epoch number: 0
display_iters: 368


  0%|          | 0/368 [00:00<?, ?it/s]

Epoch 1.000, lr 0.00100, time 166.63
loss 6572.7516 0.1974 6572.5542, ade1 293.1707, fde1 528.3082, ade 279.1145, fde 478.5155

epoch number: 1
display_iters: 368


...

Epoch 33.000, lr 0.00010, time 162.60
loss 7459.2869 0.1896 7459.0972, ade1 252.6957, fde1 381.5049, ade 236.5711, fde 333.2677

epoch number: 33
display_iters: 368


  0%|          | 0/368 [00:00<?, ?it/s]

Epoch 34.000, lr 0.00010, time 162.91
loss 7304.5381 0.1880 7304.3501, ade1 251.7997, fde1 380.2095, ade 236.2893, fde 332.3808

epoch number: 34
display_iters: 368
************************* Validation, time 112.40 *************************
loss 5534.2417 0.1936 5534.0481, ade1 247.6481, fde1 376.9258, ade 234.8548, fde 328.7293



  0%|          | 0/368 [00:00<?, ?it/s]

Epoch 35.000, lr 0.00010, time 275.97
loss 6401.7595 0.1914 6401.5682, ade1 250.5596, fde1 378.2934, ade 235.8819, fde 330.7509

epoch number: 35
display_iters: 368
Epoch 36.000, lr 0.00010, time 163.08
loss 6141.0460 0.1923 6140.8537, ade1 245.3643, fde1 371.6467, ade 234.5219, fde 329.0718

************************* Validation, time 119.83 *************************
loss 5509.7776 0.1925 5509.5851, ade1 245.1316, fde1 371.2634, ade 233.8586, fde 327.8026

